In [76]:
%load_ext autoreload
%autoreload 2

In [78]:
import datetime as dt
from session import VevenSession


async def mass_get_cookies(credentials: Dict[str, str], event_url):
    sessions = {}
    for un, pw in credentials.items():
        print('User:', un)
        Session = VevenSession(
            event_url=event_url,
            username=un, password=pw
        )
        await Session.get_cookies()
        sessions[un] = Session
    return sessions


def timed_mass_register(trigger, sessions):
    while dt.datetime.now() < trigger:
        time.sleep(0.01)
    
    responses = {}
    print('\nStarting registers. Time:', dt.datetime.now())
    for un, Session in sessions.items():
        t0 = time.time()
        res = Session.register()
        if res.status_code == 201:
            print('{} registered at appx. {}\nTime elapsed: {}s'.format(
                un, dt.datetime.now(), (time.time() - t0)))
        else:
            res = Session.register()
            print('{} failed to register. Retry code: {}\nTime elapsed: {}s'.format(
                un, res.status_code, (time.time() - t0)))
        responses[Session.username] = res
        
    return responses

In [79]:
credentials = {
    'markskag': 'pw'
}

sessions = await mass_get_cookies(credentials, 'https://omega.ntnu.no/events/show/518')
timed_mass_register(dt.datetime.now() + dt.timedelta(seconds=5), sessions)

User: markskag
Fetching new cookies with webdriver.
New cookies fetched.
Valid cookies: True

Starting registers. Time: 2020-09-09 20:02:48.267850
markskag registered at appx. 2020-09-09 20:02:48.406246
Time elapsed: 0.13809800148010254s


{'markskag': <Response [201]>}